<a href="https://colab.research.google.com/github/Hyeonwid/AIT_DL2023/blob/main/Final_CarlDavidLyricPrj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install datasets
!pip3 install transformers
!pip3 install xformers
!pip3 install --upgrade accelerate
!pip3 install evaluate
!pip3 install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.0 MB/s eta 0:00

In [ ]:
import numpy as np
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TFGPT2LMHeadModel, AutoConfig, TextStreamer, TrainingArguments, pipeline, Trainer, DataCollatorForLanguageModeling, TextDataset, DataCollatorForLanguageModeling

#, AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the fine-tuned model
save_directory = "drive/MyDrive/beatles_4epochs"

loaded_model = GPT2LMHeadModel.from_pretrained(save_directory)
loaded_tokenizer = GPT2Tokenizer.from_pretrained(save_directory)

Mounted at /content/drive


In [ ]:
seed_text = "I can't believe"
input_ids = loaded_tokenizer.encode(seed_text, return_tensors='pt')
output = loaded_model.generate(input_ids,
               max_length = 200, # max words of generated text 
               num_beams = 5, # increases output likelihood, keeps num of word sequences and chooses the best at the end
               no_repeat_ngram_size  = 2, # faults repetition of phrases, even if it would result in a better "fluency score"
               early_stopping = True)
prediction = loaded_tokenizer.decode(output[0], skip_special_tokens=True)
prediction

"hi\nI'm not sure what to say, but I know that I'm in love with you\nYou're the only one I've ever known\nAnd I don't know what else I can say\n\n\n'Cause I love you so much, I just can't wait to see what you do with me\nBaby, what do you want me to do?\nOh, baby, you're so beautiful, it's like you've never seen me before\nIt's just a dream come true\nBut I want you to be my lover\n(Baby)\nOoh, ooh-ooh!\n\nLook at me, look at you! (Oh) Oh, yeah, oh-oh\nDon't touch me like that\nJust like I said, 'cause I wanna be your best friend\nAww, that's what I like about you, isn't it? (You know) (I wanna) be yours, too\nIsn't that what we"

In [ ]:
# Load the lyrics dataset
dataset = load_dataset("koudeheld/beatles_lyrics", split='train')

# Filter the dataset for lyrics by the specified artist
artist_lyrics = dataset['lyrics']

train_split = 0.8
test_split = 0.1

train_ind = int(train_split * (len(artist_lyrics)))
test_ind = int((1 - test_split) * (len(artist_lyrics)))

X_train = (artist_lyrics[:train_ind])
# X_valid = (artist_lyrics[train_ind:test_ind])
# X_test = (artist_lyrics[test_ind:])
X_test = (artist_lyrics[train_ind:])


artist_lyrics_text = "\n".join(X_train)
# valid_text = "\n".join(X_valid)
test_text = "\n".join(X_test)


In [ ]:
# Perplexity
from tqdm import tqdm

encodings = loaded_tokenizer(test_text, return_tensors="pt")

max_length = loaded_model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = loaded_model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

 92%|█████████▏| 24/26 [03:37<00:18,  9.06s/it]


In [ ]:
# Perplexity
ppl

tensor(5.3419)

In [ ]:
# Load the GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

In [ ]:
dataset_files = "beatles.txt"

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=dataset_files,
    block_size=128  # Adjust the block size according to your dataset
)

# Define the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set mlm=True if your dataset contains masked language modeling
)
training_args = TrainingArguments(
    output_dir='./author_model',
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    save_total_limit=1,
)

# Define the Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=352, training_loss=2.116647893732244, metrics={'train_runtime': 3515.8374, 'train_samples_per_second': 0.4, 'train_steps_per_second': 0.1, 'total_flos': 91974795264000.0, 'train_loss': 2.116647893732244, 'epoch': 4.0})

In [ ]:
# Save the fine-tuned model
save_directory = "beatles_4epochs"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('beatles_4epochs/tokenizer_config.json',
 'beatles_4epochs/special_tokens_map.json',
 'beatles_4epochs/vocab.json',
 'beatles_4epochs/merges.txt',
 'beatles_4epochs/added_tokens.json')

In [ ]:
from evaluate import load
bertscore = load("bertscore")

X_test_prefix = []
for elt in X_test:
  X_test_prefix.append(" ".join(elt.split(" ")[:5]))

predictions = []
for phrase in X_test_prefix:
  input_ids = tokenizer.encode(phrase, return_tensors='pt')
  output = model.generate(input_ids,
                max_length = 200, # max words of generated text 
                num_beams = 5, # increases output likelihood, keeps num of word sequences and chooses the best at the end
                no_repeat_ngram_size  = 2, # faults repetition of phrases, even if it would result in a better "fluency score"
                early_stopping = True)
  predictions.append(tokenizer.decode(output[0], skip_special_tokens=True))


In [ ]:
references = X_test
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased", lang="en")
results

{'precision': [0.7445630431175232,
  0.7875722646713257,
  0.7535228133201599,
  0.7277059555053711,
  0.7496730089187622,
  0.7919864654541016,
  0.7590459585189819,
  0.7841107249259949,
  0.7349120378494263,
  0.7645444869995117,
  0.6321452856063843,
  0.733690619468689,
  0.735466718673706,
  0.7399103045463562,
  0.7122339606285095,
  0.786553680896759,
  0.7756639122962952,
  0.758624255657196],
 'recall': [0.8094819188117981,
  0.788813591003418,
  0.7799925804138184,
  0.6938122510910034,
  0.7487930655479431,
  0.6576477885246277,
  0.7643789052963257,
  0.7110496759414673,
  0.7383253574371338,
  0.7851817011833191,
  0.704975962638855,
  0.7571793794631958,
  0.7169095277786255,
  0.7668697834014893,
  0.6820694804191589,
  0.6795876026153564,
  0.759478747844696,
  0.7889288663864136],
 'f1': [0.7756664752960205,
  0.7881924510002136,
  0.7665292024612427,
  0.7103551030158997,
  0.7492327690124512,
  0.7185924649238586,
  0.7617030739784241,
  0.7457951307296753,
  0.7366

In [ ]:
ref = list(map(lambda x : x.split(), X_test))
tests = list(map(lambda x : x.split(), predictions))

from nltk.translate.bleu_score import sentence_bleu
for i, test in enumerate(tests):
  print('BLEU score for test' + str(i) + '-> {}'.format(sentence_bleu(ref, test)))

BLEU score for test0-> 0.15483079870066502
BLEU score for test1-> 0.16577377236021726
BLEU score for test2-> 0.19632679614835538
BLEU score for test3-> 0.0072751178566594415
BLEU score for test4-> 0.2019281091276053
BLEU score for test5-> 0.0028864882651961064
BLEU score for test6-> 0.17779417167338324
BLEU score for test7-> 0.024377529435777475
BLEU score for test8-> 0.041610597986537406
BLEU score for test9-> 0.15428552128294337
BLEU score for test10-> 0.059025906675627834
BLEU score for test11-> 0.11727621882505872
BLEU score for test12-> 0.1461358125964144
BLEU score for test13-> 0.15569020709095116
BLEU score for test14-> 0.1879090832830776
BLEU score for test15-> 0.030467653185229548
BLEU score for test16-> 0.2083217673644468
BLEU score for test17-> 0.18868166031774028


In [ ]:
sentiment_analysis = pipeline("sentiment-analysis")

texts = []
for prediction in predictions:
  texts.append(prediction[:512])
for lyrics in X_test:
  texts.append(lyrics[:512])

results = sentiment_analysis(texts)

sentiments = []
scores = []

for result in results:
  sentiments.append(result['label'])
  scores.append(result['score'])

for i in range(len(scores)):
  print(f"Text {i % 18} sentiment: {sentiments[i]}, Score: {scores[i]}")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Text 0 sentiment: POSITIVE, Score: 0.9997898936271667
Text 1 sentiment: NEGATIVE, Score: 0.9955527186393738
Text 2 sentiment: POSITIVE, Score: 0.9975892305374146
Text 3 sentiment: NEGATIVE, Score: 0.9896299242973328
Text 4 sentiment: POSITIVE, Score: 0.9995619654655457
Text 5 sentiment: POSITIVE, Score: 0.9441961646080017
Text 6 sentiment: POSITIVE, Score: 0.9996829032897949
Text 7 sentiment: POSITIVE, Score: 0.9959532022476196
Text 8 sentiment: POSITIVE, Score: 0.9994283318519592
Text 9 sentiment: POSITIVE, Score: 0.9997885823249817
Text 10 sentiment: POSITIVE, Score: 0.9969080090522766
Text 11 sentiment: POSITIVE, Score: 0.9998486042022705
Text 12 sentiment: POSITIVE, Score: 0.9990554451942444
Text 13 sentiment: POSITIVE, Score: 0.9929460883140564
Text 14 sentiment: POSITIVE, Score: 0.9961601495742798
Text 15 sentiment: POSITIVE, Score: 0.9978983402252197
Text 16 sentiment: POSITIVE, Score: 0.9582606554031372
Text 17 sentiment: POSITIVE, Score: 0.9990540146827698
Text 0 sentiment: PO